In [2]:
import aif360
from aif360.datasets import AdultDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing.reweighing import Reweighing
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
 
import numpy as np
import pandas as pd
START_BOLD = '\033[1m'
END_BOLD = '\033[0m'

In [3]:
column_names = ['age', 'workclass', 'fnlwgt', 'education',
    'education-num', 'marital-status', 'occupation', 'relationship',
    'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income-per-year']

dataset_path = Path(aif360.__file__).parent / 'data' / 'raw' / 'adult' / 'adult.data'
original_df = pd.read_csv(dataset_path, names=column_names)

In [4]:
original_df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income-per-year
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


# TASK 1

## Preprocess

In [5]:
# Using as base the aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions.load_preproc_data_adult

def load_preproc_data_adult(protected_attributes=None):
    min_privileged_age = 35
    max_privileged_age = 55
    def custom_preprocessing(df):

        def is_in_privileged_age(x):
            if x > min_privileged_age and x < max_privileged_age:
                return 1.0
            else:
                return 0.0

        def group_edu(x):
            if x <= 5:
                return '<6'
            elif x >= 13:
                return '>12'
            else:
                return x

        def group_race(x):
            if x == "White":
                return 1.0
            else:
                return 0.0

        # Limit education range
        df['education years'] = df['education-num'].apply(lambda x: group_edu(x))
        df['education years'] = df['education years'].astype('category')

        # Rename income variable
        df['income binary'] = df['income-per-year']
        df['income binary'] = df['income binary'].replace(to_replace='>50K.', value='>50K', regex=True)
        df['income binary'] = df['income binary'].replace(to_replace='<=50K.', value='<=50K', regex=True)

        # Recode sex, race and binarize age
        df['sex'] = df['sex'].replace({'Female': 0.0, 'Male': 1.0})
        df['race'] = df['race'].apply(lambda x: group_race(x))
        df['age'] = df['age'].apply(lambda x: is_in_privileged_age(x))

        return df


    XD_features = ['age', 'education years', 'sex', 'race', 'hours-per-week']
    D_features = ['age', 'race', 'sex'] if protected_attributes is None else protected_attributes
    Y_features = ['income binary']
    X_features = list(set(XD_features)-set(D_features))
    categorical_features = ['education years']

    # privileged classes
    all_privileged_classes = {"age": [1.0],
                              "race": [1.0],
                              "sex": [1.0]}

    # protected attribute maps
    all_protected_attribute_maps = {"age": {1.0: f'Between {min_privileged_age} and {max_privileged_age}', 0.0: f'Not in between {min_privileged_age} and {max_privileged_age}'}, 
                                    "race": {1.0: 'White', 0.0: 'Non-white'}, 
                                    "sex": {1.0: 'Male', 0.0: 'Female'}}

    return AdultDataset(
        label_name=Y_features[0],
        favorable_classes=['>50K', '>50K.'],
        protected_attribute_names=D_features,
        privileged_classes=[all_privileged_classes[x] for x in D_features],
        instance_weights_name=None,
        categorical_features=categorical_features,
        features_to_keep=X_features+Y_features+D_features,
        na_values=['?'],
        metadata={'label_maps': [{1.0: '>50K', 0.0: '<=50K'}],
                  'protected_attribute_maps': [all_protected_attribute_maps[x]
                                for x in D_features]},
        custom_preprocessing=custom_preprocessing)

In [6]:
dataset = load_preproc_data_adult(['age', 'sex'])

In [7]:
dataset.convert_to_dataframe()[0]

,age,race,sex,hours-per-week,education years=6,education years=7,education years=8,education years=9,education years=10,education years=11,education years=12,education years=<6,education years=>12,income binary
0,0.0,0.0,1.0,40.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,50.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,1.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,1.0,0.0,1.0,40.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,30.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0.0,1.0,0.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
48838,1.0,1.0,1.0,40.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
48839,0.0,1.0,0.0,40.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
48840,0.0,1.0,1.0,20.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
train_test_valid_split = 0.7
test_valid_split = 0.5

train_ds, valid_test_ds = dataset.split([train_test_valid_split], shuffle=True)
validation_ds, test_ds = valid_test_ds.split([test_valid_split], shuffle=True)

## Classification

In [24]:
scale = StandardScaler()
X_train = scale.fit_transform(train_ds.features)
y_train = train_ds.labels.ravel()
w_train = train_ds.instance_weights.ravel()

dtmod = DecisionTreeClassifier()
dtmod.fit(X_train, y_train, sample_weight=w_train)
pos_ind = np.where(dtmod.classes_ == train_ds.favorable_label)[0][0]

In [25]:
validation_ds_pred = validation_ds.copy(deepcopy=True)
X_valid = scale.transform(validation_ds_pred.features)
validation_ds_pred.scores = dtmod.predict_proba(X_valid)[:,pos_ind].reshape(-1,1)

test_ds_pred = test_ds.copy(deepcopy=True)
X_test = scale.transform(test_ds_pred.features)
test_ds_pred.scores = dtmod.predict_proba(X_test)[:,pos_ind].reshape(-1,1)

In [26]:
num_thresh = 99 
ba_arr = np.zeros(num_thresh)
class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
for idx, class_thresh in enumerate(class_thresh_arr):
    
    fav_inds = validation_ds_pred.scores > class_thresh
    validation_ds_pred.labels[fav_inds] = validation_ds_pred.favorable_label
    validation_ds_pred.labels[~fav_inds] = validation_ds_pred.unfavorable_label
    
    classified_metric_orig_valid = ClassificationMetric(validation_ds,
                                             validation_ds_pred)
    
    ba_arr[idx] = (classified_metric_orig_valid.true_positive_rate()
                   +classified_metric_orig_valid.true_negative_rate()) /2

best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
best_class_thresh = class_thresh_arr[best_ind]

## Results

In [27]:
fav_inds = test_ds_pred.scores > best_class_thresh
test_ds_pred.labels[fav_inds] = test_ds_pred.favorable_label
test_ds_pred.labels[~fav_inds] = test_ds_pred.unfavorable_label

metric_test = ClassificationMetric(test_ds, test_ds_pred)

balanced_accuracy = (metric_test.true_negative_rate() + metric_test.true_positive_rate()) / 2
print(f"Balanced accuracy for {START_BOLD}classifier{END_BOLD}: {round(balanced_accuracy, 4)}")

Balanced accuracy for classifier: 0.7209


# TASK 2

In [26]:
very_privileged_groups = [{'sex': 1, 'age': 1}]
slightly_privileged_groups = [{'sex': 0, 'age': 1}, {'sex': 1, 'age': 0}]
privileged_groups = very_privileged_groups + slightly_privileged_groups
unprivileged_groups = [{'sex': 0, 'age': 0}]

def print_dataset_metrics(dataset, dataset_name):

    metric_very_privileged = BinaryLabelDatasetMetric(dataset, 
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=very_privileged_groups)

    metric_slightly_privileged = BinaryLabelDatasetMetric(dataset, 
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=slightly_privileged_groups)

    metric_privileged = BinaryLabelDatasetMetric(dataset, 
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)


    print(f"{START_BOLD}{dataset_name}{END_BOLD} metrics")
    print("Difference in mean outcomes between unprivileged and very privileged groups = %f" % metric_very_privileged.mean_difference())
    print("Difference in mean outcomes between unprivileged and slightly privileged groups = %f" % metric_slightly_privileged.mean_difference())
    print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_privileged.mean_difference())



def print_classifier_metrics(base_ds, prediction_ds, best_class_thresh, classifier_name):
    fav_inds = prediction_ds.scores > best_class_thresh
    prediction_ds.labels[fav_inds] = prediction_ds.favorable_label
    prediction_ds.labels[~fav_inds] = prediction_ds.unfavorable_label

    metric_test = ClassificationMetric(base_ds, prediction_ds,
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)

    balanced_accuracy = (metric_test.true_negative_rate() + metric_test.true_positive_rate()) / 2
    statistical_parity_difference = metric_test.statistical_parity_difference()
    disparate_impact = metric_test.disparate_impact()
    average_odds_difference = metric_test.average_odds_difference()
    equal_opportunity_difference = metric_test.equal_opportunity_difference()
    theil_index = metric_test.theil_index()

    print(f"Balanced accuracy for {START_BOLD}{classifier_name}{END_BOLD}: {round(balanced_accuracy, 4)}")
    print(f"Statistical parity difference for {START_BOLD}{classifier_name}{END_BOLD}: {round(statistical_parity_difference, 4)}")
    print(f"Disparate impact for {START_BOLD}{classifier_name}{END_BOLD}: {round(disparate_impact, 4)}")
    print(f"Average odds difference for {START_BOLD}{classifier_name}{END_BOLD}: {round(average_odds_difference, 4)}")
    print(f"Equal opportunity difference for {START_BOLD}{classifier_name}{END_BOLD}: {round(equal_opportunity_difference, 4)}")
    print(f"Theil index for {START_BOLD}{classifier_name}{END_BOLD}: {round(theil_index, 4)}")

## Fairness Metrics

In [29]:
print_dataset_metrics(train_ds, "Unfair Dataset")

Unfair Dataset metrics
Difference in mean outcomes between unprivileged and very privileged groups = -0.373313
Difference in mean outcomes between unprivileged and slightly privileged groups = -0.128641
Difference in mean outcomes between unprivileged and privileged groups = -0.217569


In [30]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
               privileged_groups=very_privileged_groups)
RW.fit(train_ds)
fair_train_ds = RW.transform(train_ds)

In [31]:
print_dataset_metrics(fair_train_ds, "Fair Dataset")

Fair Dataset metrics
Difference in mean outcomes between unprivileged and very privileged groups = 0.000000
Difference in mean outcomes between unprivileged and slightly privileged groups = 0.042403
Difference in mean outcomes between unprivileged and privileged groups = 0.026991


## Fair Classifier

In [32]:
fair_scale = StandardScaler()
fair_X_train = fair_scale.fit_transform(fair_train_ds.features)
fair_y_train = fair_train_ds.labels.ravel()
fair_w_train = fair_train_ds.instance_weights

fair_dtmod = DecisionTreeClassifier()
fair_dtmod.fit(fair_X_train, fair_y_train, sample_weight=fair_w_train)

DecisionTreeClassifier()

In [33]:
fair_test_ds_pred = test_ds.copy(deepcopy=True)
fair_X_test = fair_scale.transform(fair_test_ds_pred.features)
fair_y_test = fair_test_ds_pred.labels
fair_test_ds_pred.scores = fair_dtmod.predict_proba(fair_X_test)[:,pos_ind].reshape(-1, 1)

## Results

In [34]:
print_classifier_metrics(test_ds, test_ds_pred, best_class_thresh, "classifier")

Balanced accuracy for classifier: 0.7209
Statistical parity difference for classifier: -0.4032
Disparate impact for classifier: 0.1313
Average odds difference for classifier: -0.4169
Equal opportunity difference for classifier: -0.5358
Theil index for classifier: 0.1244


In [35]:
print_classifier_metrics(test_ds, fair_test_ds_pred, best_class_thresh, "fair classifier")

Balanced accuracy for fair classifier: 0.6623
Statistical parity difference for fair classifier: -0.0582
Disparate impact for fair classifier: 0.8135
Average odds difference for fair classifier: 0.0006
Equal opportunity difference for fair classifier: -0.0175
Theil index for fair classifier: 0.1654


# Task 3

## Initial Cross Tabulation
Calculate a simple cross tabulation of the age and sex identifiers for the data set.
We begin by binarizing the age column for easier tabulation.

In [10]:
def is_in_privileged_age(x):
    min_privileged_age = 35
    max_privileged_age = 55
    if x > min_privileged_age and x < max_privileged_age:
            return 1.0
    else:
        return 0.0

In [11]:
df = original_df
df['age'] = df['age'].apply(lambda x: is_in_privileged_age(x))

In [12]:
pd.crosstab(df['age'], df['sex'], margins=True)

sex,Female,Male,All
age,,,
0.0,6908,12525,19433
1.0,3863,9265,13128
All,10771,21790,32561


## Applying LDP
We implement a simple randomizer to apply local privacy to the sensitive datasets considered above

In [13]:
import random
import math

def rand_resp(x, p, q):
        toss = random.random()
        if x == 0:
            y = 0 if toss <= q else 1
        else:
            y = 1 if toss <= p else 0
        return y

# Randomized response implementation
def apply_local_privacy(df, p, q):

    df['priv_sex'] = df['sex'].apply(lambda x: rand_resp(x, p, q))

    #Binarize age first before applying local privacy
    df['age'] = df['age'].apply(lambda x: is_in_privileged_age(x))
    df['priv_age'] = df['age'].apply(lambda x: rand_resp(x, p, q))

# P and Q value generator for a specific epsilon value
def get_p_q(epsilon):
    p = math.exp(epsilon)/(1+math.exp(epsilon))
    return p, p

We compare different epsilon values for the local privacy function

In [14]:
priv_age = df['age'].apply(lambda x: 1 if x == 1 else 0).values
males = df['sex'].apply(lambda x: 1 if x == ' Male' else 0).values

n_priv_age = np.sum(priv_age)
n_males = np.sum(males)
n_people = len(priv_age)

In [15]:
def apply_rand_resp(truth, p=0.75, q=0.75):
    return np.array([rand_resp(x, p, q) for x in truth])

def estimate(responses, p=0.75, q=0.75):
    n_people = len(responses)
    n_reported = np.sum(responses)
    return (n_reported/n_people + q - 1)/(p+q-1)*n_people

In [16]:
epsilons = [1, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3]
for x in epsilons:
    epsilon = x
    p, q = get_p_q(epsilon)
    print(f"For {epsilon:.3f}-LDP we set p={p}, q={q}.")
    
    priv_age_responses = apply_rand_resp(priv_age, p, q)
    n_est_priv_age = estimate(priv_age_responses, p, q)
    
    error = round(abs((n_priv_age - n_est_priv_age)/(n_priv_age) * 100), 2)

    print("------------------------------------------------------------------")
    print(f"There is an estimated {n_est_priv_age:.0f} people of privileged age.")
    print(f"This is very close to the actual number of {n_priv_age} people of priviliged age.")
    print(f"With and error of {error}%\n")
    
    male_responses = apply_rand_resp(males, p, q)
    n_est_males = estimate(male_responses, p, q)
    error = round(abs((n_males - n_est_males)/(n_males) * 100), 2)
    
    print(f"There is an estimated {n_est_males:.0f} males.")
    print(f"This is very close to the actual number of {n_males} males.")
    print(f"With and error of {error}%\n\n")

For 1.000-LDP we set p=0.7310585786300049, q=0.7310585786300049.
------------------------------------------------------------------
There is an estimated 13133 people of privileged age.
This is very close to the actual number of 13128 people of priviliged age.
With and error of 0.04%

There is an estimated 21856 males.
This is very close to the actual number of 21790 males.
With and error of 0.3%


For 1.250-LDP we set p=0.7772998611746912, q=0.7772998611746912.
------------------------------------------------------------------
There is an estimated 12850 people of privileged age.
This is very close to the actual number of 13128 people of priviliged age.
With and error of 2.12%

There is an estimated 21923 males.
This is very close to the actual number of 21790 males.
With and error of 0.61%


For 1.500-LDP we set p=0.8175744761936437, q=0.8175744761936437.
------------------------------------------------------------------
There is an estimated 13107 people of privileged age.
This is v

## Second Cross Tabulation

In [17]:
apply_local_privacy(df, 0.88, 0.88)
print(pd.crosstab(df['age'], df['sex'], margins=True))
pd.crosstab(df['priv_age'], df['priv_sex'], margins=True)

sex   Female   Male    All
age                       
0.0    10771  21790  32561
All    10771  21790  32561


priv_sex,0,1,All
priv_age,,,
0,3477,25163,28640
1,457,3464,3921
All,3934,28627,32561


## Implementation of Task 1 with LDP
We use the same data loader as task 1 but add the local privacy function and apply it to the sensitive values

In [18]:
def load_preproc_data_adult(protected_attributes=None):
    min_privileged_age = 35
    max_privileged_age = 55
    p = 0.88 
    q = 0.88
    def custom_preprocessing(df):

        def is_in_privileged_age(x):
            if x > min_privileged_age and x < max_privileged_age:
                return 1.0
            else:
                return 0.0

        def group_edu(x):
            if x <= 5:
                return '<6'
            elif x >= 13:
                return '>12'
            else:
                return x

        def group_race(x):
            if x == "White":
                return 1.0
            else:
                return 0.0
    
        def rand_resp(x, p, q):
            toss = random.random()
            if x == 0:
                y = 0 if toss <= q else 1
            else:
                y = 1 if toss <= p else 0
            return y

        # Limit education range
        df['education years'] = df['education-num'].apply(lambda x: group_edu(x))
        df['education years'] = df['education years'].astype('category')

        # Rename income variable
        df['income binary'] = df['income-per-year']
        df['income binary'] = df['income binary'].replace(to_replace='>50K.', value='>50K', regex=True)
        df['income binary'] = df['income binary'].replace(to_replace='<=50K.', value='<=50K', regex=True)

        # Recode sex, race and binarize age
        df['sex'] = df['sex'].replace({'Female': 0.0, 'Male': 1.0})
        df['race'] = df['race'].apply(lambda x: group_race(x))
        df['age'] = df['age'].apply(lambda x: is_in_privileged_age(x))

        # Apply local privacy to sensitive values
        df['sex'] = df['sex'].apply(lambda x: rand_resp(x, p, q))
        df['age'] = df['age'].apply(lambda x: rand_resp(x, p, q))

        return df


    XD_features = ['age', 'education years', 'sex', 'race', 'hours-per-week']
    D_features = ['age', 'race', 'sex'] if protected_attributes is None else protected_attributes
    Y_features = ['income binary']
    X_features = list(set(XD_features)-set(D_features))
    categorical_features = ['education years']

    # privileged classes
    all_privileged_classes = {"age": [1.0],
                              "race": [1.0],
                              "sex": [1.0]}

    # protected attribute maps
    all_protected_attribute_maps = {"age": {1.0: f'Between {min_privileged_age} and {max_privileged_age}', 0.0: f'Not in between {min_privileged_age} and {max_privileged_age}'}, 
                                    "race": {1.0: 'White', 0.0: 'Non-white'}, 
                                    "sex": {1.0: 'Male', 0.0: 'Female'}}

    return AdultDataset(
        label_name=Y_features[0],
        favorable_classes=['>50K', '>50K.'],
        protected_attribute_names=D_features,
        privileged_classes=[all_privileged_classes[x] for x in D_features],
        instance_weights_name=None,
        categorical_features=categorical_features,
        features_to_keep=X_features+Y_features+D_features,
        na_values=['?'],
        metadata={'label_maps': [{1.0: '>50K', 0.0: '<=50K'}],
                  'protected_attribute_maps': [all_protected_attribute_maps[x]
                                for x in D_features]},
        custom_preprocessing=custom_preprocessing)

In [19]:
private_dataset = load_preproc_data_adult(['age', 'sex'])
private_dataset.convert_to_dataframe()[0]

,age,race,sex,hours-per-week,education years=6,education years=7,education years=8,education years=9,education years=10,education years=11,education years=12,education years=<6,education years=>12,income binary
0,0.0,0.0,1.0,40.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,50.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,1.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,1.0,0.0,1.0,40.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,30.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0.0,1.0,1.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
48838,1.0,1.0,1.0,40.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
48839,0.0,1.0,0.0,40.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
48840,1.0,1.0,1.0,20.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Next we replicate the training-test split used before with the new data frame

In [20]:
train_test_valid_split = 0.7
test_valid_split = 0.5

train_ds, valid_test_ds = private_dataset.split([train_test_valid_split], shuffle=True)
validation_ds, test_ds = valid_test_ds.split([test_valid_split], shuffle=True)

Retrain of the classifier with private dataset

In [21]:
scale = StandardScaler()
X_train = scale.fit_transform(train_ds.features)
y_train = train_ds.labels.ravel()
w_train = train_ds.instance_weights.ravel()

dtmod = DecisionTreeClassifier()
dtmod.fit(X_train, y_train, sample_weight=w_train)
pos_ind = np.where(dtmod.classes_ == train_ds.favorable_label)[0][0]

validation_ds_pred = validation_ds.copy(deepcopy=True)
X_valid = scale.transform(validation_ds_pred.features)
validation_ds_pred.scores = dtmod.predict_proba(X_valid)[:,pos_ind].reshape(-1,1)

test_ds_pred = test_ds.copy(deepcopy=True)
X_test = scale.transform(test_ds_pred.features)
test_ds_pred.scores = dtmod.predict_proba(X_test)[:,pos_ind].reshape(-1,1)

num_thresh = 99 
ba_arr = np.zeros(num_thresh)
class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
for idx, class_thresh in enumerate(class_thresh_arr):
    
    fav_inds = validation_ds_pred.scores > class_thresh
    validation_ds_pred.labels[fav_inds] = validation_ds_pred.favorable_label
    validation_ds_pred.labels[~fav_inds] = validation_ds_pred.unfavorable_label
    
    classified_metric_orig_valid = ClassificationMetric(validation_ds,
                                             validation_ds_pred)
    
    ba_arr[idx] = (classified_metric_orig_valid.true_positive_rate()
                   +classified_metric_orig_valid.true_negative_rate()) /2

best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
best_class_thresh = class_thresh_arr[best_ind]

## Results

In [22]:
fav_inds = test_ds_pred.scores > best_class_thresh
test_ds_pred.labels[fav_inds] = test_ds_pred.favorable_label
test_ds_pred.labels[~fav_inds] = test_ds_pred.unfavorable_label

metric_test = ClassificationMetric(test_ds, test_ds_pred)

balanced_accuracy = (metric_test.true_negative_rate() + metric_test.true_positive_rate()) / 2
print(f"Balanced accuracy for {START_BOLD}private classifier{END_BOLD}: {round(balanced_accuracy, 4)}")

Balanced accuracy for private classifier: 0.71


There is a negative impact on the classifier using locally privatized data compared to the original classifier as it lowered the overall accuracy of the model.

# Task 4

Consider the same fairness metric and fairness mitigation method as in (2). Create a fair version of the private classifier; we will refer to is as private+fair classifier.
Assume, you’re an auditor that has access to the real sensitive values of Age and Sex. Using the real values of Age and Sex, measure the fairness of the private+fair classifier, and compare it to that of the fair classifier. Draw conclusions.

In [31]:
very_privileged_private_groups = [{'sex': 1, 'age': 1}]
slightly_privileged_private_groups = [{'sex': 0, 'age': 1}, {'sex': 1, 'age': 0}]
privileged_private_groups = very_privileged_private_groups + slightly_privileged_private_groups
unprivileged_private_groups = [{'sex': 0, 'age': 0}]

Starting from the private dataset created we are going to create a fair version of it. Firstly, we need to reweight it just as in part 2. 

In [32]:
RW = Reweighing(unprivileged_groups=unprivileged_private_groups,
               privileged_groups=very_privileged_private_groups)
RW.fit(private_dataset)
fair_private_train_ds = RW.transform(private_dataset)

From the results below you can see that the private dataset had a high difference between the unpriviledged and all other privileged groups. 

In [33]:
print_dataset_metrics(private_dataset, "Private Dataset")

Private Dataset metrics
Difference in mean outcomes between unprivileged and very privileged groups = -0.284230
Difference in mean outcomes between unprivileged and slightly privileged groups = -0.106058
Difference in mean outcomes between unprivileged and privileged groups = -0.169229


Nonetheless, this effect is setoff after reweighting the dataset.

In [34]:
print_dataset_metrics(fair_private_train_ds, "Fair Private Dataset")

Fair Private Dataset metrics
Difference in mean outcomes between unprivileged and very privileged groups = 0.000000
Difference in mean outcomes between unprivileged and slightly privileged groups = 0.025744
Difference in mean outcomes between unprivileged and privileged groups = 0.016616


In [35]:
train_test_valid_split = 0.7
test_valid_split = 0.5

train_ds, valid_test_ds = dataset.split([train_test_valid_split], shuffle=True)
validation_ds, test_ds = valid_test_ds.split([test_valid_split], shuffle=True)

In [36]:
fair_private_scale = StandardScaler()
fair_private_X_train = fair_private_scale.fit_transform(train_ds.features)
fair_private_y_train = train_ds.labels.ravel()
fair_private_w_train = train_ds.instance_weights

fair_private_dtmod = DecisionTreeClassifier()
fair_private_dtmod.fit(fair_private_X_train, fair_private_y_train, sample_weight=fair_private_w_train)
pos_ind = np.where(fair_private_dtmod.classes_ == train_ds.favorable_label)[0][0]

In [39]:
fair_private_test_ds_pred = test_ds.copy(deepcopy=True)
fair_private_X_test = fair_private_scale.transform(fair_private_test_ds_pred.features)
fair_private_y_test = fair_private_test_ds_pred.labels
fair_private_test_ds_pred.scores = fair_private_dtmod.predict_proba(fair_private_X_test)[:,pos_ind].reshape(-1, 1)

The Fair Private Classifier achieves higher balanced accuracy (72.1% vs. 66.2%), indicating better overall performance. However, this comes at the expense of fairness, as shown by significantly worse fairness metrics such as statistical parity difference (-0.452 vs. -0.058) and disparate impact (0.121 vs. 0.813). Additionally, the Fair Private Classifier demonstrates larger biases in equal opportunity (-0.544 vs. -0.017) and average odds (-0.451 vs. 0.0006). Despite its better information distribution (Theil index: 0.115 vs. 0.165), the private classifier's increased disparity in treating different groups raises concerns about its suitability for sensitive applications where fairness is critical.

In [41]:
print_classifier_metrics(test_ds, fair_private_test_ds_pred, best_class_thresh, "Fair Private classifier")

Balanced accuracy for Fair Private classifier: 0.7205
Statistical parity difference for Fair Private classifier: -0.4522
Disparate impact for Fair Private classifier: 0.1216
Average odds difference for Fair Private classifier: -0.4515
Equal opportunity difference for Fair Private classifier: -0.5444
Theil index for Fair Private classifier: 0.1151
